# ***Eğitim Performansı***

## ***Hava durumu modeli***

Bu ödev için, geçmiş sıcaklık veri kümesine yeniden bakacaksınız. Öncelikle, Sıcaklık verilerini [Kaggle](https://www.kaggle.com/budincsevity/szeged-weather/data)'dan indirin.

- Hedef değişkeninizin görünür sıcaklık ve sıcaklık arasındaki fark olduğu bir doğrusal regresyon modeli oluşturun. Açıklayıcı değişkenler olarak nem ve rüzgar hızı kullanın. Şimdi, modelinizi OLS kullanarak tahmin edin. R-kare ve ayarlanmış R-kare değerleri nelerdir? Tatminkar olduklarını düşünüyor musunuz? Açıklayın.

- Yukarıdaki modele nem ve rüzgar hızı etkileşimini dahil edin ve OLS'yi kullanarak modeli tahmin edin. Şimdi, bu modelin R-kare değeri nedir? Bu model bir öncekine göre gelişti mi?

- İlk modele ek açıklayıcı değişken olarak görünürlük ekleyin ve tahmin edin. R-kare arttı mı? Ayarlanmış R-kare değeri ne oldu? Tabloda ortaya çıkan farklılıkları, ayarlanmış R-kare içindeki iyileşme açısından etkileşim terimi ve görünürlük ile karşılaştırın. Hangisi daha kullanışlı?

- AIC ve BIC puanlarına göre yukarıdaki üç modelden en iyisini seçin. Mentor ile gerekçenizi tartışarak seçiminizi doğrulayın.

In [1]:
import pandas as pd
import numpy as np
# grafiksel araclar
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid")

from sklearn import linear_model
import statsmodels.api as sm

# uyarilari dikkate alma
import warnings
warnings.filterwarnings('ignore')

# pandas varsayilan olarak cok sayida sutun veya satir varsa tumunu gostermez
# bu nedenle 100 sutun ve satir gostermesi icin
pd.options.display.max_rows = 100
pd.options.display.max_columns = 100

# virgulden sonra belirli sayida basamak gosterimi icin 
pd.options.display.float_format = '{:,.3f}'.format

# font tanimlamalari
title_font = {'family': 'times new roman', 'color': 'darkred','weight': 'bold','size': 14}
axis_font  = {'family': 'times new roman', 'color': 'darkred','weight': 'bold','size': 14}

# temiz veri setini dataframe icerisine yukle
weather = pd.read_csv('data/weatherHistory.csv')

In [2]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96453 entries, 0 to 96452
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Formatted Date            96453 non-null  object 
 1   Summary                   96453 non-null  object 
 2   Precip Type               95936 non-null  object 
 3   Temperature (C)           96453 non-null  float64
 4   Apparent Temperature (C)  96453 non-null  float64
 5   Humidity                  96453 non-null  float64
 6   Wind Speed (km/h)         96453 non-null  float64
 7   Wind Bearing (degrees)    96453 non-null  float64
 8   Visibility (km)           96453 non-null  float64
 9   Loud Cover                96453 non-null  float64
 10  Pressure (millibars)      96453 non-null  float64
 11  Daily Summary             96453 non-null  object 
dtypes: float64(8), object(4)
memory usage: 8.8+ MB


- Sonuclar dataframe olusturularak kaydedilecek.

In [68]:
# sonuclari tutacak df
results_df = pd.DataFrame(index=[1, 2, 3],
                          columns=['rsquared','rsquared_adj', 'fvalue', 'aic', 'bic'])
results_df.index.name = 'model'
# sonuclari kaydeden fonksiyon
def save_results(results, model_n):
    results_df.loc[model_n, 'rsquared'] = results.rsquared
    results_df.loc[model_n, 'rsquared_adj'] = results.rsquared_adj
    results_df.loc[model_n, 'fvalue'] = results.fvalue
    results_df.loc[model_n, 'aic'] = results.aic
    results_df.loc[model_n, 'bic'] = results.bic

---
### ***1. Model***

In [69]:
# aciklayici degiskenler listesi
expl_vars = ['Humidity',  'Wind Speed (km/h)']

# hedef degisken
weather['Temp_Diff'] = weather['Temperature (C)'] - weather['Apparent Temperature (C)'] 

# hedef degiskeni
Y = weather['Temp_Diff']
# aciklayici degiskenler
X = weather[expl_vars]

# bias eklendi
X = sm.add_constant(X)

results = sm.OLS(Y, X).fit()

# results nesnesinin metod ve degiskenleri
# dir(results)

# 1 inci model sonuclarini kaydedelim
save_results(results,1)

# 1 inci model sonuclari
results_df.loc[1,:]

- R-kare ve ayarlanmış R-kare değerleri 0.28, modelin hedef degisken varyansini aciklama orani %28, sonuc tatmin edici degildir.

---
### ***2. Model***

In [71]:
weather['Humidity_WindSpeed'] = weather['Humidity'] * weather['Wind Speed (km/h)']

# aciklayici degiskenler listesi
expl_vars = ['Humidity',  'Wind Speed (km/h)', 'Humidity_WindSpeed']

# hedef degisken
weather['Temp_Diff'] = weather['Temperature (C)'] - weather['Apparent Temperature (C)'] 

# hedef degiskeni
Y = weather['Temp_Diff']
# aciklayici degiskenler
X = weather[expl_vars]

# bias eklendi
X = sm.add_constant(X)

results = sm.OLS(Y, X).fit()

# 2 inci model sonuclarini kaydedelim
save_results(results,2)

# 2 inci model sonuclari
results_df.loc[2,:]

rsquared             0.341
rsquared_adj         0.341
fvalue          16,656.182
aic            333,393.099
bic            333,431.006
Name: 2, dtype: object

- R-kare ve ayarlanmış R-kare değerleri 0.34 e yukseldi, modelin hedef degisken varyansini aciklama orani %34, sonuc tatmin edici degildir.

---
### ***3. Model***

In [72]:
# aciklayici degiskenler listesi
expl_vars = ['Humidity',  'Wind Speed (km/h)', 'Humidity_WindSpeed', 'Visibility (km)']

# hedef degisken
weather['Temp_Diff'] = weather['Temperature (C)'] - weather['Apparent Temperature (C)'] 

# hedef degiskeni
Y = weather['Temp_Diff']
# aciklayici degiskenler
X = weather[expl_vars]

# bias eklendi
X = sm.add_constant(X)

results = sm.OLS(Y, X).fit()

# 3 inci model sonuclarini kaydedelim
save_results(results,3)

# 2 inci model sonuclari
results_df

,rsquared,rsquared_adj,fvalue,aic,bic
model,,,,,
1,0.288,0.288,"19,489.463","340,916.925","340,945.355"
2,0.341,0.341,"16,656.182","333,393.099","333,431.006"
3,0.364,0.363,"13,770.948","330,082.577","330,129.961"


- R-kare ve ayarlanmış R-kare değerleri 0.36 e yukseldi, modelin hedef degisken varyansini aciklama orani %36. R-kare degerlerine gore en kullanisli model budur.

- AIC ve BIC puanlarina gore de 3 uncu model en iyisidir.